Performs RAG (Retrieval-Augmented Generation) using the organized course data generated by ChatGPT
<br> Explanation on RAG: https://www.ibm.com/docs/en/watsonx/saas?topic=solutions-retrieval-augmented-generation

In [ ]:
# import packages
from dotenv import load_dotenv
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_ibm import WatsonxLLM
from langchain.vectorstores import FAISS
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watsonx_ai import Credentials

In [ ]:
# Load the organized textbook data
textbook_extracted_path = r"C:\Users\ediso\OneDrive\Desktop\IBM Call for Code\rita-cfc-2024\ai\course-prep\textbook-extracted\nan_math_5th_2nd_extracted.txt"

with open(textbook_extracted_path, "r", encoding="utf-8") as file:
    extracted_text = file.read()    

In [ ]:
# Create a RecursiveCharacterTextSplitter object to split the text into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # Maximum number of characters in each chunk
    chunk_overlap=200,     # Number of characters that overlap between consecutive chunks
    length_function=len    # Function to measure the length of chunks
)

texts = text_splitter.split_text(extracted_text)

# Display the first few chunks to ensure proper splitting
# for i, chunk in enumerate(texts[:5]):
#     print(f"Chunk {i+1}:\n{chunk}\n")

In [ ]:
# Convert Text Chunks into Embeddings (dense vector representation of the text that capture semantic information)

# Initialize the embedding model using Model on HuggingFace
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Initialize FAISS (Facebook AI Similarity Search) vector store, converting raw text chunks into embeddings
faiss_store = FAISS.from_texts(texts, embedding_model)

# save_path = r'C:\Users\ediso\OneDrive\Desktop\IBM Call for Code\rita-cfc-2024\ai\course-prep\RAG\vector-stores'

# TODO relative path
# Define the save path and the name for the vector store
save_path = r'C:\Users\ediso\OneDrive\Desktop\IBM Call for Code\rita-cfc-2024\ai\course-prep\RAG\vector-stores'
vector_store_name = 'nan_math_5th_2nd_vector_store'

full_save_path = os.path.join(save_path, vector_store_name)
os.makedirs(full_save_path, exist_ok=True)

# Save FAISS vector store to disk with a name
faiss_store.save_local(full_save_path)

# Load FAISS store from disk
faiss_store = FAISS.load_local(full_save_path, embedding_model, allow_dangerous_deserialization=True)

# Create a retriever chain
retriever = faiss_store.as_retriever()

In [ ]:
# Load sensitive info
load_dotenv()
API_KEY = os.getenv('API_KEY')
URL = os.getenv('URL')
PROJECT_ID = os.getenv('PROJECT_ID')

In [ ]:
# Initialize WatsonX LLM Interface

credentials = Credentials.from_dict({
    'url': URL,
    'apikey': API_KEY
})

params = {
    GenParams.MAX_NEW_TOKENS: 4095,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1.2
}

# Initialize the LLM model
llm = WatsonxLLM(
    model_id=ModelTypes.LLAMA_3_70B_INSTRUCT.value,
    params=params,
    # credentials=credentials,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=PROJECT_ID
)

# Define the QA chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
# 
# Define the query
query = "跟我說更多關於課程1-1的內容，全部使用繁體中文"

# Get the response using the query embedding
response = qa.invoke({"query": query})

print(response['result'])

In [ ]:
# Define the query
query = "給我一些關於課程1-1的例題，全部使用繁體中文"

# Get the response using the query embedding
response = qa.invoke({"query": query})

print(response['result'])

In [ ]:
# Define the query
query = "給我一些關於課程1-1的例題，全使用繁體中文"

# Get the response using the query embedding
response = qa.invoke({"query": query})

print(response['result'])

In [ ]:
# Define the query
query = "給我一些關於課程1-1的例題，結合一些生活情境，全部使用繁體中文"

# Get the response using the query embedding
response = qa.invoke({"query": query})

print(response['result'])

In [ ]:
# Define the query
query = "給我一些關於課程1-1的例題，全使用繁體中文"

# Get the response using the query embedding
response = qa.invoke({"query": query})

print(response['result'])